In [15]:
from langchain_community.document_loaders import PyPDFLoader

#loading pdf
pdf_loader = PyPDFLoader("rag_data/*.pdf")
pages = pdf_loader.load()

In [16]:
from langchain_community.document_loaders.csv_loader import CSVLoader
# loading csv
csv_loader = CSVLoader(file_path='rag_data/FA2024.csv')
rows = csv_loader.load()

'VLSI for Comm and ML\n"4 CREDITS for Lec 002"; Prerequisites for Lec 002: (EECS 351 and (EECS312 or EECS 370)) or graduate standing or\npermission of instructor. \n-------------------------------------------------------------------------------------\n34184 P  W LEC 003 TTH 9-1030AM 1008 EECS Grbic\nElectromagnetic Metamaterials\n"3 CREDITS for Lec 003"; Prerequisites for Lec 003:330 \n-------------------------------------------------------------------------------------\n34185 P  W LEC 004 MW 1030-12PM 1311 EECS Scott\nStatistical Learning Theory\n"3 CREDITS for Lec 004"; Prerequisites for Lec 004: EECS 545 or 553 \n-------------------------------------------------------------------------------------\n35530 P  W LEC 005 MW 12-130PM 1670 BEYSTER Oymak\nLarge Language Models\n"3 CREDITS for Lec 005"; Prerequisites for Lec 011:EECS 445 or 453 or 505 or 545 or 553 \n-------------------------------------------------------------------------------------\n35517 P  W LEC 006 T 330-630PM 1025 GG

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,
chunks = text_splitter.split_documents(pages)

In [24]:
chunks

[Document(metadata={'source': 'rag_data/Umich_2024_EECS_class_schedule.pdf', 'page': 0}, page_content='VLSI for Comm and ML\n"4 CREDITS for Lec 002"; Prerequisites for Lec 002: (EECS 351 and (EECS312 or EECS 370)) or graduate standing or\npermission of instructor. \n-------------------------------------------------------------------------------------\n34184 P  W LEC 003 TTH 9-1030AM 1008 EECS Grbic\nElectromagnetic Metamaterials\n"3 CREDITS for Lec 003"; Prerequisites for Lec 003:330 \n-------------------------------------------------------------------------------------\n34185 P  W LEC 004 MW 1030-12PM 1311 EECS Scott\nStatistical Learning Theory\n"3 CREDITS for Lec 004"; Prerequisites for Lec 004: EECS 545 or 553 \n-------------------------------------------------------------------------------------\n35530 P  W LEC 005 MW 12-130PM 1670 BEYSTER Oymak\nLarge Language Models\n"3 CREDITS for Lec 005"; Prerequisites for Lec 011:EECS 445 or 453 or 505 or 545 or 553 \n-----------------------

In [22]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [26]:
from langchain_chroma import Chroma
from langchain_voyageai import VoyageAIEmbeddings

embeddings = VoyageAIEmbeddings(
    voyage_api_key="pa-m_h441WnBOr-nKGPOKaeswmo8Dte1rPtCt_uq3AMrKU", model="voyage-2"
)

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory="./chroma_db")

batch size None


In [27]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retreived_docs = retriever.invoke("What time are EECS classes next semester?")

In [28]:
retreived_docs

[Document(metadata={'page': 14, 'source': 'rag_data/Umich_2024_EECS_class_schedule.pdf'}, page_content='course meets the first 7 weeks of the term.  \n-------------------------------------------------------------------------------------\n37665 A RW MID 993 TH 6-8PM ARR Kapritsos\nMidterm for EECS 498-003 is on Oct 17. \n-------------------------------------------------------------------------------------\n499 Adv Directed Study 1.00-4.00 ADVISORY, ENFORCED  \n  R IND +  ARR ARR\n507 Intro Embed Sys Res 3.00-4.00 ADVISORY  \n26955 P  W LEC 001 TTH 3-430PM 1311 EECS Dick\n542 Adv Top Comp Vision 3.00 ADVISORY  \nOpen to all by waitlisting and subsequent override permission only. Most overrides, if granted, expect to be\nprocessed during the summer in July and August, though some may be granted earlier. There is no need to contact the\ninstructor for override permission, but do respond to any emails from the instructor.  CSE Override System \n \nThese requests are for students interested 